In [1]:
from network import Network
from conv import Conv
from dense import Dense
from activation import Relu
from metrics import log_loss, log_loss_grad, softmax
import math
import numpy as np
import random

In [2]:
random.seed(0)
FILE = "fashion-mnist_train.csv"
with open(f"cnn/images/{FILE}") as f:
    examples = f.read().strip().split("\n")[1:]

train, valid, test = [], [], []
valid_cutoff = .8
test_cutoff = .9

for example in examples:
    label, pixels = example.split(",", 1)
    label = int(label)
    if label > 4:
        continue
    pixels = [int(p) for p in pixels.split(",")]
    data = np.asarray(pixels, dtype="int32")
    data = data.reshape((1,28,28))
    # Scale values from -1 to 1
    data = ((data / 255) - .5) / .5
    target = np.zeros((1, 5))
    target[0,label] = 1
    row = (data, target, )

    split = random.random()
    if split > test_cutoff:
        test.append(row)
    elif split > valid_cutoff:
        valid.append(row)
    else:
        train.append(row)

In [3]:
class TestNet(Network):
    def __init__(self):
        self.conv1 = Conv(input_channels=1, output_channels=1, kernel_x=3, kernel_y=3)
        self.conv2 = Conv(input_channels=3, output_channels=1, kernel_x=3, kernel_y=3)
        self.dense = Dense(input_size=26 ** 2, output_size=5)
        self.relu = Relu()
        self.last_input = None

        super().__init__()

    def forward(self, x):
        self.last_input = x.copy()
        x = self.conv1.forward(x)
        x = self.relu.forward(x)
        #x = self.conv2.forward(x)
        x = x.reshape(1, math.prod(x.shape))
        x = self.dense.forward(x)
        return x

    def backward(self, grad, lr):
        prev_hidden = self.conv1.hidden.copy().reshape(1, math.prod(self.conv1.hidden.shape))
        grad = self.dense.backward(grad, lr, prev_hidden)
        grad = grad.reshape(self.conv1.hidden.shape)
        grad = self.relu.backward(grad, lr)
        #grad = self.conv2.backward(grad, lr)
        grad = self.conv1.backward(grad, lr, self.last_input)

In [5]:
net = TestNet()
lr = 5e-4
epochs = 20

for epoch in range(epochs):
    epoch_loss = 0
    for i, img in enumerate(train):
        image, target = img
        batch = net.forward(image)

        grad = log_loss_grad(softmax(batch), target)
        epoch_loss += log_loss(softmax(batch), target)
        net.backward(grad, lr)

    print(f"Epoch {epoch} loss: {epoch_loss / len(train)}")
    match = np.zeros(len(valid))
    for i, img in enumerate(valid):
        image, target = img
        valid_pred = net.forward(image)
        match[i] = np.argmax(valid_pred) == np.argmax(target)

    print(f"Epoch {epoch} valid accuracy: {match.sum() / match.shape[0]}")

Epoch 0 loss: 0.6002991842880065
Epoch 0 valid accuracy: 0.8156312625250501
Epoch 1 loss: 0.5126062929364192
Epoch 1 valid accuracy: 0.8326653306613226
Epoch 2 loss: 0.4987298621915705
Epoch 2 valid accuracy: 0.8383433533734135
Epoch 3 loss: 0.49129139505268804
Epoch 3 valid accuracy: 0.844689378757515
Epoch 4 loss: 0.48577001326740304
Epoch 4 valid accuracy: 0.8490313961255845
Epoch 5 loss: 0.4808721834301965
Epoch 5 valid accuracy: 0.8510354041416166
Epoch 6 loss: 0.47617989685355
Epoch 6 valid accuracy: 0.85437541750167
Epoch 7 loss: 0.4715504889534388
Epoch 7 valid accuracy: 0.8563794255177021
Epoch 8 loss: 0.4669031977313645
Epoch 8 valid accuracy: 0.8570474281897128
Epoch 9 loss: 0.46221382787541765
Epoch 9 valid accuracy: 0.8577154308617234
Epoch 10 loss: 0.4574739006979759
Epoch 10 valid accuracy: 0.857381429525718
Epoch 11 loss: 0.45268621320560287
Epoch 11 valid accuracy: 0.8580494321977288
Epoch 12 loss: 0.4478865637696985
Epoch 12 valid accuracy: 0.8600534402137608
Epoch 13